In [251]:
import pandas as pd
import numpy as np

#import csv from database
movies = pd.read_csv('./database/movies_local.csv')
movies = movies.sort_values('movie')
movies = movies.set_index('movie')

#rename all columns
movies.columns = ['indication_year', 'number_indications', 'victories']

In [252]:
#convet year into numbers
movies['indication_year'] = movies['indication_year'].astype(int)
movies.head()

,indication_year,number_indications,victories
movie,,,
"$1,000 a Minute",1936,1,0
'38',1987,1,0
'Crocodile' Dundee,1987,1,0
'Round Midnight,1987,2,1
(A) Torzija [(A) Torsion],2004,1,0


# Gerando Algumas estatisticas

In [253]:
def percentage_victories(indications, victories):
    return (victories * 100) / indications
movies['percentage_victories'] = np.vectorize(percentage_victories)(movies['number_indications'], movies['victories'])

In [254]:
movies.head()

,indication_year,number_indications,victories,percentage_victories
movie,,,,
"$1,000 a Minute",1936,1,0,0.0
'38',1987,1,0,0.0
'Crocodile' Dundee,1987,1,0,0.0
'Round Midnight,1987,2,1,50.0
(A) Torzija [(A) Torsion],2004,1,0,0.0


In [352]:
bdteste = movies.iloc[-10:,:].copy()
bdteste

,indication_year,number_indications,victories,percentage_victories
movie,,,,
Zorba the Greek,1965,7,3,42.857143
Zus & Zo,2003,1,0,0.000000
images,1973,1,0,0.000000
"sex, lies, and videotape",1990,1,0,0.000000
the accountant,2002,1,1,100.000000
the end,1996,1,0,0.000000
tom thumb,1959,1,1,100.000000
À Nous la Liberté,1933,1,0,0.000000
Ådalen '31,1970,1,0,0.000000


In [353]:
import datetime
now = datetime.datetime.now()

print(str(now.hour)+':'+str(now.minute)+':'+str(now.second))

bkp = []

14:51:1


In [392]:
import requests
import json
api_key = 'aa043c2ae33192f63fcc150bb97816b7'


def budg_imdb_rev(mvid):
    if mvid and mvid != 0:
        url = "https://api.themoviedb.org/3/movie/"+str(mvid)+"?api_key="+api_key+"&language=en-US"

        payload = "{}"

        response = requests.request("GET", url, data=payload)
        data = json.loads(response.text)
        genres = [x['name'] for x in data['genres']]
        production_countries = [x['iso_3166_1'] for x in data['production_countries']]
        return  np.array([data['imdb_id'], data['budget'], data['revenue'], genres, production_countries], dtype=object)
    
    return  np.array([False, False, False, False, False], dtype=object)

def find_movie(mv, yr):
    
    now = datetime.datetime.now()
    url = "https://api.themoviedb.org/3/search/movie?api_key="+api_key+"&language=en-US&query="+mv
    url= url+"&primary_release_year="+str(yr-1)
    payload = "{}"

    response = requests.request("GET", url, data=payload)
    data = json.loads(response.text)
    
    print(str(now.hour)+':'+str(now.minute)+':'+str(now.second))
    print(mv + ' | ' +str(data['total_results']))
    
    if len(data['results']) == 1:
        return  data['results'][0]['id']
    
    elif data['results'] and data['total_results'] > 1:
        lmmv = []
        
        for mvs in data['results']:
            if mvs['title'] == mv:
                lmmv.append(mvs)
                
        if len(lmmv) == 1:
            return  lmmv[0]['id']
        elif len(lmmv) > 1:
            return  False
        
        return  False
   
    return  False

In [399]:
#divide movies database for analisis

moviesdv=[]

for x in range(1,120):
    moviesdv.append(movies.iloc[int(len(movies.index.values)/120)*(x-1):int(len(movies.index.values)/120)*x,:].copy())

len(moviesdv[0])

39

In [ ]:
for df in moviesdv:
    print('-----------------------------------------------')
    df['tmdb_id'] =  np.vectorize(find_movie)(df.index.values, df['indication_year'])

-----------------------------------------------
15:41:54
$1,000 a Minute | 1
15:41:54
$1,000 a Minute | 1
15:41:55
'38' | 1
15:41:56
'Crocodile' Dundee | 1
15:41:57
'Round Midnight | 1
15:41:58
(A) Torzija [(A) Torsion] | 0
15:41:58
...And Justice for All | 1
15:41:59
10 | 10
15:41:59
102 Dalmatians | 1
15:42:0
12 | 11
15:42:1
12 Angry Men | 1
15:42:1
12 Monkeys | 1
15:42:2
12 Years a Slave | 1
15:42:3
127 Hours | 1
15:42:3
12:01 PM | 1
15:42:4
13 Hours: The Secret Soldiers of Benghazi | 1
15:42:5
13th | 2
15:42:5
140 Days under the World | 1
15:42:6
1776 | 1
15:42:7
1848 | 0
15:42:7
1941 | 1
15:42:9
20 Feet from Stardom | 1
15:42:9
20,000 Leagues under the Sea | 1
15:42:10
2001: A Space Odyssey | 1
15:42:11
2010 | 1
15:42:11
20th Century Women | 1
15:42:12
21 Grams | 1
15:42:13
24-Hour Alert | 1
15:42:13
3 Misses | 0
15:42:14
3:10 to Yuma | 1
15:42:15
3rd Ave. El | 1
15:42:15
4 Little Girls | 1
15:42:16
4.1 Miles | 1
15:42:16
42nd Street | 2
15:42:17
45 Years | 1
15:42:18
5 Broken Cam

15:44:54
Always in My Heart | 1
15:44:55
Ama Girls | 1
15:44:56
Amadeus | 1
15:44:56
Amarcord | 0
15:44:57
Amarcord | 0
15:44:58
Amazon | 3
15:44:59
Amazon Diary | 1
15:44:59
America America | 4
15:45:0
American Beauty | 1
15:45:0
American Dream | 2
15:45:1
American Gangster | 1
15:45:2
American Graffiti | 1
15:45:3
American History X | 1
15:45:3
American Hustle | 1
15:45:4
American Shoeshine | 1
15:45:4
American Sniper | 1
15:45:5
American Splendor | 1
15:45:6
Americas in Transition | 0
15:45:6
Amistad | 1
15:45:7
Amores Perros | 1
15:45:8
Amour | 24
15:45:8
Amphibious Fighters | 1
15:45:9
Amy | 5
15:45:10
Amélie | 1
15:45:10
An Affair to Remember | 1
-----------------------------------------------
15:45:11
An American Dream | 1
15:45:12
An American Dream | 1
15:45:12
An American Tail | 1
15:45:13
An American Werewolf in London | 1
15:45:14
An American in Paris | 1


In [360]:
moviesdv[0]

,indication_year,number_indications,victories,percentage_victories,tmdb_id
movie,,,,,
Zorba the Greek,1965,7,3,42.857143,10604
Zus & Zo,2003,1,0,0.000000,21896
images,1973,1,0,0.000000,0
"sex, lies, and videotape",1990,1,0,0.000000,1412
the accountant,2002,1,1,100.000000,36733
the end,1996,1,0,0.000000,0
tom thumb,1959,1,1,100.000000,56931
À Nous la Liberté,1933,1,0,0.000000,0
Ådalen '31,1970,1,0,0.000000,87208


In [ ]:
movies.to_csv('./database/movies_TMDB_PI1.csv')

In [ ]:
now = datetime.datetime.now()

print(str(now.hour)+':'+str(now.minute)+':'+str(now.second))

In [ ]:

movies[['imdb_id','budget','revenue', 'genres', 'production_countries']] = pd.DataFrame(list(movies['tmdb_id'].apply(budg_imdb_rev)), index=movies.index)


In [ ]:
movies.to_csv('./database/movies_TMDB_PI2.csv')

In [ ]:
now = datetime.datetime.now()

print(str(now.hour)+':'+str(now.minute)+':'+str(now.second))

In [ ]:
#https://developers.themoviedb.org/3/movies/get-movie-details
#https://github.com/richardasaurus/imdb-pie#get_title_credits
#https://market.mashape.com/marcalencc/metacritic#
movies.head()